### Imports

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from IPython.display import display

from sklearn.naive_bayes import GaussianNB

from pandas.api.types import is_string_dtype

from sklearn.ensemble import RandomForestClassifier

from fastai_utils import add_datepart, train_cats, proc_df

import os
import math

%matplotlib inline


#### Carregar o dataset e limpar os dados

In [2]:
df_raw_enem_2018 = pd.read_csv("Projeto Educação/microdados_enem2018/DADOS/MICRODADOS_ENEM_2018.csv", 
                               low_memory=True, 
                               sep=';',  
                               encoding="latin_1")

In [3]:
df_enem_2018 = df_raw_enem_2018[['SG_UF_RESIDENCIA',
                                 'NU_IDADE',
                                 'TP_SEXO',
                                 'TP_COR_RACA',
                                 'TP_NACIONALIDADE',
                                 'SG_UF_PROVA',
                                 'NU_NOTA_CN',
                                 'NU_NOTA_CH', 
                                 'NU_NOTA_LC',
                                 'NU_NOTA_MT',
                                 'Q027',
                                 'Q006']]
df_enem_2018

,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,SG_UF_PROVA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,Q027,Q006
0,DF,44.0,M,1,0,GO,NaN,NaN,NaN,NaN,A,A
1,MA,23.0,F,3,1,MA,521.0,568.7,513.1,480.3,A,B
2,SP,23.0,F,1,1,RS,NaN,NaN,NaN,NaN,A,C
3,BA,26.0,F,3,2,BA,NaN,NaN,NaN,NaN,A,B
4,BA,20.0,M,0,1,BA,NaN,419.9,345.8,NaN,A,B
...,...,...,...,...,...,...,...,...,...,...,...,...
5513742,RJ,56.0,F,1,1,RJ,NaN,NaN,NaN,NaN,C,D
5513743,AM,20.0,F,3,1,AM,435.6,556.8,460.8,528.1,A,B
5513744,SP,17.0,F,3,1,SP,NaN,NaN,NaN,NaN,A,C
5513745,SP,17.0,F,1,1,SP,524.9,496.5,512.5,474.1,C,D


In [4]:
df_enem_2018 = df_enem_2018.rename(columns={'Q027':'TipoEscola', 'Q006':'FaixaRenda'})
df_enem_2018

,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_COR_RACA,TP_NACIONALIDADE,SG_UF_PROVA,NU_NOTA_CN,NU_NOTA_CH,NU_NOTA_LC,NU_NOTA_MT,TipoEscola,FaixaRenda
0,DF,44.0,M,1,0,GO,NaN,NaN,NaN,NaN,A,A
1,MA,23.0,F,3,1,MA,521.0,568.7,513.1,480.3,A,B
2,SP,23.0,F,1,1,RS,NaN,NaN,NaN,NaN,A,C
3,BA,26.0,F,3,2,BA,NaN,NaN,NaN,NaN,A,B
4,BA,20.0,M,0,1,BA,NaN,419.9,345.8,NaN,A,B
...,...,...,...,...,...,...,...,...,...,...,...,...
5513742,RJ,56.0,F,1,1,RJ,NaN,NaN,NaN,NaN,C,D
5513743,AM,20.0,F,3,1,AM,435.6,556.8,460.8,528.1,A,B
5513744,SP,17.0,F,3,1,SP,NaN,NaN,NaN,NaN,A,C
5513745,SP,17.0,F,1,1,SP,524.9,496.5,512.5,474.1,C,D


In [5]:
def display_all(df):
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
        print(df)

In [6]:
raça = {1:"Branca",
        2:"Preta",
        3:"Parda",
        4:"Amarela",
        5:"Indígena"}

In [7]:
nacionalida = { 0:"Não informado",
                1:"Brasileiro(a)",
                2:"Brasileiro(a) Naturalizado(a)",
                3:"Estrangeiro(a)",
                4:"Brasileiro(a) Nato(a), nascido(a) no exterior"}

In [8]:
TipoEscola = {  'A':"Somente em escola pública.", #0
                'B':"Parte em escola pública e parte em escola privada SEM bolsa de estudo integral.", #1
                'C':"Parte em escola pública e parte em escola privada COM bolsa de estudo integral.",#2
                'D':"Somente em escola privada SEM bolsa de estudo integral.",#3
                'E':"Somente em escola privada COM bolsa de estudo integral.", #4
                'F':"Não frequentei a escola"} #5

In [9]:
FaixaRenda = {  'A':"Nenhuma renda.",
                'B':"Até R$ 954,00.",
                'C':"De R$ 954,01 até R$ 1.431,00.",
                'D':"De R$ 1.431,01 até R$ 1.908,00.",
                'E':"De R$ 1.908,01 até R$ 2.385,00.",
                'F':"De R$ 2.385,01 até R$ 2.862,00.",
                'G':"De R$ 2.862,01 até R$ 3.816,00.",
                'H':"De R$ 3.816,01 até R$ 4.770,00.",
                'I':"De R$ 4.770,01 até R$ 5.724,00.",
                'J':"De R$ 5.724,01 até R$ 6.678,00.",
                'K':"De R$ 6.678,01 até R$ 7.632,00.",
                'L':"De R$ 7.632,01 até R$ 8.586,00.",
                'M':"De R$ 8.586,01 até R$ 9.540,00.",
                'N':"De R$ 9.540,01 até R$ 11.448,00.",
                'O':"De R$ 11.448,01 até R$ 14.310,00.",
                'P':"De R$ 14.310,01 até R$ 19.080,00.",
                'Q':"Mais de R$ 19.080,00."}

In [10]:
gnb = GaussianNB()

In [19]:
dic_acertos = {}
tentativa = 1
def testa_acertos():
    global tentativa
    acerto = 1-(sum([y != y_chapeu for y_chapeu, y in zip(y_pred, y_test)])/len(y_pred))
    dic_acertos[tentativa] = [acerto, X.columns]
    tentativa += 1

## Primeira tentativa! gnb.fit

In [13]:
X, y, _ = proc_df(df_enem_2018, 'TipoEscola')

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=55)

In [15]:
y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [20]:
testa_acertos()
dic_acertos

{1: [0.7466773140639624,
  Index(['SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO', 'TP_COR_RACA',
         'TP_NACIONALIDADE', 'SG_UF_PROVA', 'NU_NOTA_CN', 'NU_NOTA_CH',
         'NU_NOTA_LC', 'NU_NOTA_MT', 'FaixaRenda', 'NU_IDADE_na',
         'NU_NOTA_CN_na', 'NU_NOTA_CH_na', 'NU_NOTA_LC_na', 'NU_NOTA_MT_na'],
        dtype='object')]}

### gnb df_enem_2018[['FaixaRenda', 'SG_UF_RESIDENCIA', 'TP_SEXO','TP_COR_RACA']]

In [22]:
X, y, _ = proc_df(df_enem_2018[['FaixaRenda', 'SG_UF_RESIDENCIA', 'TP_SEXO','TP_COR_RACA', 'TipoEscola']], 'TipoEscola')

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=55)

In [24]:
y_pred = gnb.fit(X_train, y_train).predict(X_test)

In [25]:
testa_acertos()
dic_acertos

{1: [0.7466773140639624,
  Index(['SG_UF_RESIDENCIA', 'NU_IDADE', 'TP_SEXO', 'TP_COR_RACA',
         'TP_NACIONALIDADE', 'SG_UF_PROVA', 'NU_NOTA_CN', 'NU_NOTA_CH',
         'NU_NOTA_LC', 'NU_NOTA_MT', 'FaixaRenda', 'NU_IDADE_na',
         'NU_NOTA_CN_na', 'NU_NOTA_CH_na', 'NU_NOTA_LC_na', 'NU_NOTA_MT_na'],
        dtype='object')],
 2: [0.7937794065193509,
  Index(['FaixaRenda', 'SG_UF_RESIDENCIA', 'TP_SEXO', 'TP_COR_RACA'], dtype='object')]}